In [1]:
import myosuite
import gym
import skvideo.io
import numpy as np
import os

RS:> Registering Myo Envs
myoSarFingerReachFixed-v0, myoFatFingerReachFixed-v0, myoSarFingerReachRandom-v0, myoFatFingerReachRandom-v0, myoSarElbowPose1D6MFixed-v0, myoFatElbowPose1D6MFixed-v0, myoSarElbowPose1D6MRandom-v0, myoFatElbowPose1D6MRandom-v0, myoSarElbowPose1D6MExoFixed-v0, myoFatElbowPose1D6MExoFixed-v0, myoSarElbowPose1D6MExoRandom-v0, myoFatElbowPose1D6MExoRandom-v0, myoSarFingerPoseFixed-v0, myoFatFingerPoseFixed-v0, myoSarFingerPoseRandom-v0, myoFatFingerPoseRandom-v0, myoSarHandPoseFixed-v0, myoFatHandPoseFixed-v0, myoTTHandPoseFixed-v0, myoSarHandPose0Fixed-v0, myoFatHandPose0Fixed-v0, myoTTHandPose0Fixed-v0, myoSarHandPose1Fixed-v0, myoFatHandPose1Fixed-v0, myoTTHandPose1Fixed-v0, myoSarHandPose2Fixed-v0, myoFatHandPose2Fixed-v0, myoTTHandPose2Fixed-v0, myoSarHandPose3Fixed-v0, myoFatHandPose3Fixed-v0, myoTTHandPose3Fixed-v0, myoSarHandPose4Fixed-v0, myoFatHandPose4Fixed-v0, myoTTHandPose4Fixed-v0, myoSarHandPose5Fixed-v0, myoFatHandPose5Fixed-v0, myoTTHandPose5Fixed

### Add conda environment to the kernel list

python -m ipykernel install --user --name=< name of the environment >

In [2]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [ ]:
pth = '/Users/caggiano/Dropbox (Meta)/Musculoskeletal_Mujoco/mjrl_dev/mjrl_dev/agents/NPG/biomechanics/'
policy = pth+"/ElbowPose1D6MExoRandom-v0/2021-09-11_10-58-45/iterations/best_policy.pickle"

import pickle
pi = pickle.load(open(policy, 'rb'))


In [3]:
env = gym.make('myoElbowPose1D6MRandom-v0')

env.reset();


array([ 1.9886,  0.0071, -0.3068,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ])

In [27]:

config_file = "../../../myosuite/agents/config/hydra_myo_config.yaml"

with open(config_file, 'r') as f:
#     job_data = eval(f.read())
    print(f.read())


default:
    - override hydra/output: local
    - override hydra/launcher: local

# general inputs
env               :   myosuite-v1          # placeholder name, not a real env
algorithm         :   NPG
seed              :   123
sample_mode       :   trajectories
rl_num_traj       :   96
rl_num_samples    :   0                # will be ignored when sample_mode=trajectories
num_cpu           :   8
rl_num_iter       :   1051
save_freq         :   100
eval_rollouts     :   10
exp_notes         :   "Sample config for MyoSuite"

# RL parameters (all params related to PG, value function etc.)
policy_size       :   (32, 32)
init_log_std      :   -0.25
min_log_std       :   -1.0
vf_hidden_size    :   (128, 128)
vf_batch_size     :   64
vf_epochs         :   2
vf_learn_rate     :   1e-3
rl_step_size      :   0.1
rl_gamma          :   0.995
rl_gae            :   0.97

# Algorithm hyperparameters : if alg requires additional params, can be specified here (or defaults will be used)

alg_hyper_para

In [46]:
dir(agent.env)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_elapsed_steps',
 '_max_episode_steps',
 'action_space',
 'class_name',
 'close',
 'compute_reward',
 'env',
 'metadata',
 'observation_space',
 'render',
 'reset',
 'reward_range',
 'seed',
 'spec',
 'step',
 'unwrapped']

In [ ]:
"""
This is a job script for running policy gradient algorithms on gym tasks.
Separate job scripts are provided to run few other algorithms
- For DAPG see here: https://github.com/aravindr93/hand_dapg/tree/master/dapg/examples
- For model-based NPG see here: https://github.com/aravindr93/mjrl/tree/master/mjrl/algos/model_accel
"""
import warnings
warnings.filterwarnings('ignore')

from mjrl.utils.gym_env import GymEnv
from mjrl.policies.gaussian_mlp import MLP
# from mjrl.baselines.quadratic_baseline import QuadraticBaseline
from mjrl.baselines.mlp_baseline import MLPBaseline
from mjrl.algos.npg_cg import NPG
# from mjrl.algos.batch_reinforce import BatchREINFORCE
# from mjrl.algos.ppo_clip import PPO
from mjrl.utils.train_agent import train_agent
# import os
# import json
# import gym
# import time as timer
import myosuite
    
policy_size = (32, 32)
vf_hidden_size = (128, 128)
seed = 123
rl_step_size = 0.1
e = GymEnv(env)

policy = MLP(e.spec, hidden_sizes=policy_size, seed=seed, init_log_std=-0.25, min_log_std=-1.0)

baseline = MLPBaseline(e.spec, reg_coef=1e-3, batch_size=64, hidden_sizes=vf_hidden_size, \
                    epochs=2, learn_rate=1e-3)

agent = NPG(e, policy, baseline, normalized_step_size=rl_step_size, \
            seed=seed, save_logs=True)

print("========================================")
print("Starting policy learning")
print("========================================")

ts = timer.time()
train_agent(job_name='.',
            agent=agent,
            seed=seed,
            niter=1000,
            gamma=0.995,
            gae_lambda=0.97,
            num_cpu=8,
            sample_mode="trajectories",
            num_traj=96,
            num_samples=0,
            save_freq=100,
            evaluation_rollouts=10)
print("========================================")
print("Job Finished. Time taken = %f" % (timer.time()-ts))
print("========================================")


Starting policy learning
......................................................................................
ITERATION : 0 
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 1.016063 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.269289 
Iter | Stoc Pol | Mean Pol | Best (Stoc) 

[ Mon Feb 21 18:28:19 2022 ]    0 53.80 57.46 -100000000.00 
------------------  ------------
VF_error_after         0.905633
VF_error_before        0.999413
alpha                  3.18982
delta                  0.1
eval_score            57.458
eval_success          18.75
kl_dist                0.0486064
num_samples         9600
rollout_success       52.0833
running_score         53.8033
rwd_dense              0.536337
rwd_sparse            -0.735644
stoc_pol_max         650.157
stoc_pol_mean         53.8033
stoc_pol_min        -172.12
stoc_pol_std         204.256
success_percentage    5

Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.270712 
[ Mon Feb 21 18:28:36 2022 ]    7 532.88 715.72 503.93 
------------------  ------------
VF_error_after         0.064808
VF_error_before        0.0753541
alpha                  2.57654
delta                  0.1
eval_score           715.721
eval_success         100
kl_dist                0.0535965
num_samples         9600
rollout_success       97.9167
running_score        223.999
rwd_dense              5.32262
rwd_sparse            -0.175766
stoc_pol_max         741.228
stoc_pol_mean        532.876
stoc_pol_min          17.3452
stoc_pol_std         145.804
success_percentage    97.9167
surr_improvement       0.0364856
time_VF                1.63911
time_npg               0.213729
time_sampling          0.918709
time_vpg               0.0104182
------------------  ------------
...........................................................................

======= Samples Gathered  ======= | >>>> Time taken = 0.901316 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.275378 
[ Mon Feb 21 18:29:03 2022 ]   15 698.81 751.65 713.81 
------------------  ------------
VF_error_after         0.0142752
VF_error_before        0.0155524
alpha                  3.23342
delta                  0.1
eval_score           751.652
eval_success         100
kl_dist                0.0515411
num_samples         9600
rollout_success      100
running_score        484.308
rwd_dense              6.98111
rwd_sparse            -0.104878
stoc_pol_max         779.266
stoc_pol_mean        698.814
stoc_pol_min         290.269
stoc_pol_std         105.026
success_percentage   100
surr_improvement       0.0289761
time_VF                2.00469
time_npg               0.241253
time_sampling          0.903669
time_vpg               0.0139282
------------------  ------------
.....................

======= Samples Gathered  ======= | >>>> Time taken = 1.233221 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.615681 
[ Mon Feb 21 18:29:36 2022 ]   23 750.80 758.68 747.93 
------------------  --------------
VF_error_after         0.000345441
VF_error_before        0.000349691
alpha                  1.92253
delta                  0.1
eval_score           758.682
eval_success         100
kl_dist                0.0525074
num_samples         9600
rollout_success      100
running_score        630.328
rwd_dense              7.50056
rwd_sparse            -0.069266
stoc_pol_max         781.345
stoc_pol_mean        750.803
stoc_pol_min         598.458
stoc_pol_std          27.9415
success_percentage   100
surr_improvement       0.0380848
time_VF                1.73534
time_npg               0.23255
time_sampling          1.23648
time_vpg               0.010591
------------------  --------------
...............

======= Samples Gathered  ======= | >>>> Time taken = 1.335367 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.610555 
[ Mon Feb 21 18:30:12 2022 ]   31 760.71 760.47 758.79 
------------------  --------------
VF_error_after         2.80534e-05
VF_error_before        2.63213e-05
alpha                  2.13336
delta                  0.1
eval_score           760.469
eval_success         100
kl_dist                0.0500613
num_samples         9600
rollout_success      100
running_score        702.286
rwd_dense              7.5996
rwd_sparse            -0.0581379
stoc_pol_max         782.529
stoc_pol_mean        760.708
stoc_pol_min         723.081
stoc_pol_std          15.9383
success_percentage   100
surr_improvement       0.0305564
time_VF                1.92906
time_npg               0.259596
time_sampling          1.3378
time_vpg               0.00956011
------------------  --------------
.............

...

======= Samples Gathered  ======= | >>>> Time taken = 0.889150 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.276674 
[ Mon Feb 21 18:33:22 2022 ]   79 766.51 760.63 771.33 
------------------  --------------
VF_error_after         8.86796e-06
VF_error_before        8.26999e-06
alpha                  2.24692
delta                  0.1
eval_score           760.632
eval_success         100
kl_dist                0.0430999
num_samples         9600
rollout_success      100
running_score        765.897
rwd_dense              7.65754
rwd_sparse            -0.0537985
stoc_pol_max         793.436
stoc_pol_mean        766.511
stoc_pol_min         727.606
stoc_pol_std          17.3214
success_percentage   100
surr_improvement       0.0271744
time_VF                1.97253
time_npg               0.238762
time_sampling          0.891933
time_vpg               0.0118999
------------------  --------------
...........

======= Samples Gathered  ======= | >>>> Time taken = 1.124893 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.606328 
[ Mon Feb 21 18:33:52 2022 ]   87 767.83 761.09 771.33 
------------------  --------------
VF_error_after         5.63704e-06
VF_error_before        6.00855e-06
alpha                  1.80414
delta                  0.1
eval_score           761.085
eval_success         100
kl_dist                0.0476346
num_samples         9600
rollout_success      100
running_score        766.637
rwd_dense              7.67077
rwd_sparse            -0.0493568
stoc_pol_max         793.35
stoc_pol_mean        767.834
stoc_pol_min         722.058
stoc_pol_std          17.3822
success_percentage   100
surr_improvement       0.0402098
time_VF                1.79658
time_npg               0.249633
time_sampling          1.12709
time_vpg               0.0123141
------------------  --------------
.............

======= Samples Gathered  ======= | >>>> Time taken = 1.239989 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.620148 
[ Mon Feb 21 18:34:27 2022 ]   95 765.42 760.78 771.33 
------------------  --------------
VF_error_after         5.21947e-06
VF_error_before        6.24881e-06
alpha                  2.16242
delta                  0.1
eval_score           760.78
eval_success         100
kl_dist                0.0429554
num_samples         9600
rollout_success      100
running_score        767.163
rwd_dense              7.64672
rwd_sparse            -0.0516704
stoc_pol_max         793.688
stoc_pol_mean        765.425
stoc_pol_min         718.21
stoc_pol_std          17.2245
success_percentage   100
surr_improvement       0.0232164
time_VF                2.29254
time_npg               0.242119
time_sampling          1.24222
time_vpg               0.0103149
------------------  --------------
..............

======= Samples Gathered  ======= | >>>> Time taken = 1.117770 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.385547 
[ Mon Feb 21 18:35:02 2022 ]  103 767.79 761.17 771.33 
------------------  --------------
VF_error_after         4.65013e-06
VF_error_before        5.07088e-06
alpha                  1.96422
delta                  0.1
eval_score           761.174
eval_success         100
kl_dist                0.0452433
num_samples         9600
rollout_success      100
running_score        767.387
rwd_dense              7.67035
rwd_sparse            -0.0479169
stoc_pol_max         794.641
stoc_pol_mean        767.795
stoc_pol_min         727.562
stoc_pol_std          16.0969
success_percentage   100
surr_improvement       0.0231807
time_VF                1.73283
time_npg               0.24267
time_sampling          1.1198
time_vpg               0.0117159
------------------  --------------
..............

======= Samples Gathered  ======= | >>>> Time taken = 0.907014 
Performing evaluation rollouts ........
####### Gathering Samples #######
======= Samples Gathered  ======= | >>>> Time taken = 0.377744 
[ Mon Feb 21 18:35:29 2022 ]  111 768.72 760.84 771.50 
------------------  --------------
VF_error_after         4.25752e-06
VF_error_before        7.46731e-06
alpha                  2.16034
delta                  0.1
eval_score           760.843
eval_success         100
kl_dist                0.0402557
num_samples         9600
rollout_success      100
running_score        767.522
rwd_dense              7.67963
rwd_sparse            -0.0474812
stoc_pol_max         795.148
stoc_pol_mean        768.72
stoc_pol_min         734.115
stoc_pol_std          16.4057
success_percentage   100
surr_improvement       0.0533311
time_VF                1.49183
time_npg               0.25716
time_sampling          0.909057
time_vpg               0.010756
------------------  --------------
..............

In [ ]:
AngleSequence = [60, 30, 30, 60, 80, 80, 60, 30, 80, 30, 80, 60]
env.reset()
frames = []
for ep in range(len(AngleSequence)):
    print("Ep {} of {} testing angle {}".format(ep, len(AngleSequence), AngleSequence[ep]))
    env.env.target_jnt_value = [np.deg2rad(AngleSequence[int(ep)])]
    env.env.update_target()
    for _ in range(40):
        frame = env.sim.render(width=400, height=400,mode='offscreen', camera_name=None)
        frames.append(frame[::-1,:,:])
        o = env.get_obs()
        a = pi.get_action(o)[0]
        next_o, r, done, ifo = env.step(a) # take an action based on the current observation
env.close()

os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/exo_arm.mp4', np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})


In [ ]:
show_video('videos/exo_arm.mp4')